## Deep MNIST Tensorflow Tutorial

Source: https://www.tensorflow.org/get_started/mnist/pros

In this tutorial:
- Create a softmax regression function that is a model for recognizing MNIST digits, based on looking at every pixel in the image
- Use Tensorflow to train the model to recognize digits by having it "look" at thousands of examples (and run our first Tensorflow session to do so)
- Check the model's accuracy with our test data
- Build, train, and test a multilayer convolutional neural network to improve the results

### Setup

Loading MNIST Data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

URLError: <urlopen error [Errno 60] Operation timed out>

### Start TensorFlow InteractiveSession 

- TF relies on a C++ backend for computation
- Create a connection through a session

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

### Build a Softmax Regression Model

#### Placeholders

In [ ]:
#Placeholder
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

#### Variables

Before Variables can be used within a session, they must be initialized using that session. 

In [ ]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Initialize all variables at once
sess.run(tf.global_variables_initializer())

#### Predicting Class and Loss Function 

We can now implement the regression model

In [ ]:
y = tf.matmul(x,W) + b

And specify the loss function. Loss indicates how bad the model's prediction was on a single example. Here, the loss function is the cross-entropy between the target and the softmax activation function applied to the model's prediction

In [ ]:
#Note that tf.nn.softmax_cross_entropy_with_logits internally applies 
#the softmax on the model's unnormalized model prediction and sums across all classes, 
#and tf.reduce_mean takes the average over these sums.
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

### Train the Model

Since we have defined the model and loss function, tensorflow knows the entire computation graph and it can use automatic differentation to find the gradients of the loss with respect to each of the variables

In [ ]:
# Optimization algo: Steepest gradient descent with a step length of 0.5
# This line adds new operations to the computation graph. 
# These operations included ones to compute gradients, compute parameter
# update steps, and apply update steps to the parameters
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Train_step will apply the gradient descent updates to the parameters. Training the model can therefore be accomplished by repeatedly running train_step. We load 100 training examples in each training iteration. The feed dict replaces the placeholder tensors x and y with training examples

In [ ]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict = {x: batch[0], y_: batch[1]})

### Evaluate the Model

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

## Build a Multilayer Convolution Network

### Weight Initialization

Need to create a lot of weights and biases. Good practice to initialize them with a slightly positive initial bias to avoid "dead neurons"

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution and Pooling 

TF provides lots of flexibility with regards to convolution and pooling, but we will stick to the vanilla version for this tutorial. Convolutions will use a stride of one and are zero padded so that the output is the same as the input. Our pooling is plain old max pooling over 2x2 blocks.

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#### First Convolution Layer

The first layer will consist of convolution, followed by max pooling. The convolution will compute 32 features for each 5x5 patch. Its weight tensore will have a shape of [5, 5, 1, 32], where [patch_dimension, patch_dimension, input_channels, output_channels]. Plus, an additional bias vector with a component for each output channel.

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

First reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

In [ ]:
x_image = tf.reshape(x, [-1,28,28,1])

Next, convolve x_image with the weight tensor, add the bias, and apply the ReLU function, and finally max pool. The max pool method will reduce the image size to 14x14.

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#### Second Convolution Layer

In order for a deep network, we stack several layers of this type. This second layer will have 64 features for each 5x5 patch.

In [5]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

NameError: name 'weight_variable' is not defined

#### Densely Connected Layer

The image size is reduced to 7x7. We now add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#### Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during the dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#### Readout Layer

Finally, we add a layer, just like for the one layer softmax regression above.

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Train and Evaluate the Model

To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network above.

Differences:
- We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.
- We will include the additional parameter keep_prob in feed_dict to control the dropout rate.
- We will add logging to every 100th iteration in the training process.

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

The final test set accuracy after running this code should be approximately 99.2%.